In [1]:
import json
import os


In [10]:
teams_dict = {}
with open("../Data/teams.json") as f:
        teams = json.load(f)
        for team in teams:
            if team['type'] == 'club':
                teams_dict[team['wyId']] = team['name']


In [11]:
players_pos_dict = {}
with open("../Data/players.json") as f:
    players = json.load(f)
    for p in players:
        players_pos_dict[p['wyId']] = p['role']['code2']



In [12]:
matchesDict = {}

with open("../Data/matches_England.json") as f:
    matches = json.load(f)
    for m in matches:
        teams = list(m['teamsData'].keys())
        matchesDict[m['wyId']] = list(m['teamsData'].keys())


In [13]:
regionsDict = {}

regionsDict[1] = [16.5, 1800/61]
regionsDict[2] = [16.5, 4300/61]
regionsDict[3] = [16.5, 100]
regionsDict[4] = [33.25, 1800/61]
regionsDict[5] = [33.25, 4300/61]
regionsDict[6] = [33.25, 100]
regionsDict[7] = [50, 1800/61]
regionsDict[8] = [50, 4300/61]
regionsDict[9] = [50, 100]
regionsDict[10] = [66.75, 1800/61]
regionsDict[11] = [66.75, 4300/61]
regionsDict[12] = [66.75, 100]
regionsDict[13] = [83.5, 1800/61]
regionsDict[14] = [83.5, 4300/61]
regionsDict[15] = [83.5, 100]
regionsDict[16] = [100, 1800/61]
regionsDict[17] = [100, 4300/61]
regionsDict[18] = [100, 100]


In [72]:
goals = {}
e = 0
with open("../Data/events_England.json") as f:
    events = json.load(f)
    while e < len(events):
        if {'id': 101} in events[e]['tags'] and events[e]['subEventId'] in [100, 91]:
            team = events[e]['teamId']
            if team not in goals:
                goals[team] = []
            tempDict = {}
            tempDict['matchId'] = events[e]['matchId']
            tempDict['teamId'] = team
            tempDict['teamName'] = teams_dict[team]
            tempDict['playerId'] = events[e]['playerId']
            tempDict['position'] = events[e]['positions'][0]
            for r in range(1, 19):
                if (tempDict['position']['x'] <= regionsDict[r][0]) & (tempDict['position']['y'] <= regionsDict[r][1]):
                    tempDict['region'] = r
                    break
            tempDict['playingPos'] = players_pos_dict[events[e]['playerId']]
            if events[e]['teamId'] == int(matchesDict[events[e]['matchId']][0]):
                tempDict['opponentId'] = matchesDict[events[e]['matchId']][1]
            else:
                tempDict['opponentId'] = matchesDict[events[e]['matchId']][0]
            tempDict['oppenentName'] = teams_dict[int(tempDict['opponentId'])]
            tempDict['id'] = events[e]['id']
            p_index = e-1
            evnt = events[p_index]['eventName']
            passes = []
            while evnt == "Pass":
                pass_dict = {}
                pass_dict['position'] = events[p_index]['positions'][0]
                for r in range(1, 19):
                    if (pass_dict['position']['x'] <= regionsDict[r][0]) & (pass_dict['position']['y'] <= regionsDict[r][1]):
                        pass_dict['region'] = r
                        break
                pass_dict['playerId'] = events[p_index]['playerId']
                pass_dict['playingPos'] = players_pos_dict[events[p_index]['playerId']]
                p_index -= 1
                evnt = events[p_index]['eventName']
                passes.append(pass_dict)
            tempDict['passes'] = passes
            goals[team].append(tempDict)
            e += 1
        e += 1
                

In [73]:
goals_per_team = {}
for k, v in goals.items():
    passes = 0
    for goal in v:
        passes += len(goal['passes'])
    goals_per_team[k] = [len(v), passes/len(v)]

In [74]:
total_goals = 0
for team in goals:
    total_goals += len(goals[team])

In [75]:
total_goals

932

In [76]:
for k, v in goals.items():
    v = [x for x in v if len(x['passes']) > 1]
    len_network = 0
    for g in v:
        len_network += len(g['passes'])
    goals[k] = v

In [77]:
total_goals = 0
for team in goals:
    total_goals += len(goals[team])

In [78]:
total_goals

239

In [79]:
goals_per_team_filtered = {}
for k, v in goals.items():
    passes = 0
    for goal in v:
        passes += len(goal['passes'])
    goals_per_team_filtered[k] = [len(v), passes/len(v)]

In [81]:
goals_per_team

{1609: [69, 2.3333333333333335],
 1631: [50, 0.98],
 1625: [95, 2.336842105263158],
 1646: [34, 1.088235294117647],
 1610: [56, 1.1607142857142858],
 1673: [24, 1.4583333333333333],
 1623: [37, 0.8378378378378378],
 1611: [65, 1.2923076923076924],
 1624: [68, 1.161764705882353],
 1644: [40, 1.525],
 1612: [78, 1.4871794871794872],
 1627: [30, 0.7666666666666667],
 1639: [34, 0.9411764705882353],
 1633: [46, 0.782608695652174],
 1659: [43, 1.2790697674418605],
 10531: [27, 0.7407407407407407],
 1613: [37, 1.162162162162162],
 1651: [30, 1.4],
 1619: [33, 0.8181818181818182],
 1628: [36, 1.4166666666666667]}

In [80]:
goals_per_team_filtered

{1609: [22, 6.409090909090909],
 1631: [11, 3.4545454545454546],
 1625: [41, 5.0],
 1646: [8, 3.75],
 1610: [11, 5.090909090909091],
 1673: [8, 3.875],
 1623: [8, 2.75],
 1611: [16, 4.1875],
 1624: [18, 3.7222222222222223],
 1644: [8, 6.625],
 1612: [23, 4.434782608695652],
 1627: [5, 3.6],
 1639: [6, 4.333333333333333],
 1633: [9, 3.3333333333333335],
 1659: [10, 5.0],
 10531: [4, 4.0],
 1613: [8, 4.375],
 1651: [6, 6.333333333333333],
 1619: [7, 2.7142857142857144],
 1628: [10, 4.4]}

In [82]:
with open('../Data/goals.json', 'w') as f:
    json.dump(goals, f)